# <center>CMPBIO210, IB120, IB201
# <center>"Introduction to Quantitative Methods in Biology"
# <center>Lecture 5. Analytical Solutions of Differential Equations
## <center>Denis Titov

**The goal of this jupyter notebook is to introduce you to analytical methods of solving differential equations using SymPy dsolve() and introduce SymPy library in general**

In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

## Introduction to SymPy

SymPy is a python library for symbolic math calculations.  
We will be mostly concerned with using SymPy to analytically solve differential equations but SymPy has many other capabilities that we'll briefly review in the beginning.



The core idea behind SymPy is that it allows you to do symbolic calculation using symbols instead of numbers

In [ ]:
x, y = sym.symbols("x y")

In [ ]:
5*x-4*x

Why should we care about doing symbolic math on a computer when numerical calculations are so easy?  
There're at least two agruments in support of symbolic math calculations:  
- Equations can be dramatically simplified using symbolic math so that it's easier to do numerical calculations 
- Numerical calculations have errors introduced by approximation of numerical methods and by inability of computers to represent rational numbers precisely  


### Representation of numbers by a computer

The number of numbers between 0 and 1 is $\infty$ while computers have finite memory so can only represent finite numbers.  
This can lead to errors in numerical calculations that you should be familiar with.  
This is not specific to python but rather a general "feature" of computers.  
You can read more about representation of floating point number by a computer here: https://docs.python.org/3/tutorial/floatingpoint.html  
For example, the value below should be True but...

In [ ]:
0.1 + 0.1 + 0.1 == .3

The example below should be equal to zero but...

In [ ]:
(0.1+0.1+0.1+0.1+0.1+0.1+0.1+0.1+0.1+0.1-1)*1e20

If we look more closely, 0.1 is not exactly 0.1 

In [ ]:
0.1

In [ ]:
format(0.1, '.20')

This is because computers represent float64 numbers as binary fractions with relative precision of $2^{-52} \approx 2.22 \cdot 10^{-16}$.  
This is called IEEE 754 standard for representing floating-point numbers.  
Computers represent numbers as a ratio of a number from $0$ to $2^{52}$ and $2^n$ where $n$ can be $-1022$ to $1023$.  
So for 0.1 it will be $\frac{3602879701896397}{2^{55}}=\frac{3602879701896397}{36028797018963968}$

In [ ]:
format(3602879701896397 / 36028797018963968,'.20')

Max precision of float numbers in computers is called machine epsilon

In [ ]:
np.finfo(float).eps

SymPy and other libraries allow you to do exact calculations using ratios of numbers

In [ ]:
sym.Rational(1,10)+sym.Rational(1,10)+sym.Rational(1,10)==sym.Rational(3,10)

### Solving algebraic equations with SymPy solveset() and nonlinsolve()

solveset() can be used to analytically solve a large variety of algebraic equations and systems of equations

In [ ]:
x,y = sym.symbols("x y")
sym.solveset(x**2+x-3*y,y)

nonlinsolve() can be used to analytically solve systems of nonlinear equations

In [ ]:
x,y,a,b = sym.symbols("x y a b")
sym.nonlinsolve(
    [-x+a*y+(x**2)*y, b-a*y-(x**2)*y],
    [x,y]
)

### Simplifying equation with SymPy simplify()

simplify() and related function can be used to simplify various equations, which SymPy doesn't do by default.  
It is often useful to define if a particular variable or function is expected to be a real number, positive number, integer etc to allow more appropriate simplifications

In [ ]:
sym.cos(x) ** 2 + sym.sin(x) ** 2

In [ ]:
sym.simplify(sym.cos(x) ** 2 + sym.sin(x) ** 2)

### Calculating differentials with SymPy diff

In [ ]:
x = sym.symbols("x")
sym.diff(x**2 + 3 - sym.cos(x),x)

Remember that $\frac{dy(t)}{dt}=\lim_{\Delta t \to 0} \frac{y(t+\Delta t)-y(t)}{\Delta t}$ so we can also calculate differential using SymPy limit() function:

In [ ]:
x, Δt = sym.symbols("x Δt")
sym.limit(
    (((x + Δt)**2 + 3 - sym.cos(x + Δt)) - (x**2 + 3 - sym.cos(x)))/Δt,
    Δt,
    0
)

### Calculating integrals with SymPy integrate()

In [ ]:
x = sym.symbols("x")
sym.integrate(2*x + sym.sin(x))

### SymPy equations can be numerically evaluated using subs() and evalf()

In [ ]:
x, y, z = sym.symbols("x, y, z")
expr = x ** 2 + y ** 2
expr

subs() can be used to substitute variable with numbers or with other variables

In [ ]:
expr.subs({x: sym.pi, y: z ** 2})

In [ ]:
expr.subs({x: sym.pi, y: 1})

evalf() can be used to numerically evaluate the equation with desired numerical precision

In [ ]:
expr.subs({x: sym.pi, y: 1}).evalf(10)

## Analytically solving ODEs with SymPy dsolve()

**Lets use our favorite bacterial growth rate in rich media as an example:**
  
## $\frac{dN(t)}{dt}=kN(t)$  

where $t$ is time, k is some constant specific for this E.coli strain and growth conditions, $N(t)$ is number of E. coli at time $t$.  

In [ ]:
N = sym.symbols("N", cls=sym.Function)
t, k, N0 = sym.symbols("t, k, N0")

Growth_solution = sym.dsolve(sym.diff(N(t), t) - k * N(t), N(t), ics={N(0): N0})
Growth_solution

We can plot the solution using SymPy built-in plotting function.  
SymPy built-in plot function doesn't require us to make an array of $t$ like matplotlib so might be more convenient.

In [ ]:
sym.plot(
    Growth_solution.rhs.subs({N0: 1, k: sym.ln(2)}),
    Growth_solution.rhs.subs({N0: 2, k: sym.ln(2)}),
    Growth_solution.rhs.subs({N0: 4, k: sym.ln(2)}),
    (t, 0, 5),
    xlabel="t",
    ylabel="N(t)",
);

**Lets use Drug clearance by the kidneys as another example:**

## $\frac{d[Drug](t)}{dt}=-k[Drug](t)$  
  
where $t$ is time,  k is some constant describing kindey filtration rate,  $[Drug](t)$ is blood concentration of drug at time $t$ and $[Drug]_0$ is is blood concentration of drug at $t=0$. 

In [ ]:
Drug = sym.symbols("[Drug]", cls=sym.Function)
t, k, Drug_0 = sym.symbols("t, k, [Drug]_0")
Drug_clearance_solution = sym.dsolve(sym.diff(Drug(t), t) + k * Drug(t), Drug(t), ics={Drug(0): Drug_0})
Drug_clearance_solution

In [ ]:
sym.plot(
    Drug_clearance_solution.rhs.subs({Drug_0: 10, k: sym.ln(2)}),
    Drug_clearance_solution.rhs.subs({Drug_0: 30, k: sym.ln(2)}),
    Drug_clearance_solution.rhs.subs({Drug_0: 100, k: sym.ln(2)}),
    (t, 0, 5),
    xlabel="t",
    ylabel="[Drug](t)"
);

You can use this approach to calculate $[S](t)$ for any substrate as long as you know the rate equation and kinetic parameters that describe that enzyme.

**Lets solve logistic equation for population growth in the presence of limited resources as an example:**
  
## $\frac{dN(t)}{dt}=kN(t) \cdot (1-\frac{N(t)}{N_{max}})$  

where $t$ is time, k is some constant specific for this population growth, $N(t)$ is population at time $t$, $N_{max}$ is maximal population size that can be achieved under these conditions.

In [ ]:
N = sym.symbols("N", cls=sym.Function)
t, k, N0, N_max = sym.symbols("t, k, N0, N_max")
Logistic_growth_solution = sym.dsolve(sym.diff(N(t), t) - k * N(t) * (1 - N(t) / N_max), 
                                      N(t), 
                                      ics={N(0): N0})
Logistic_growth_solution.simplify()

In [ ]:
sym.plot(
    Logistic_growth_solution.rhs.subs({N0: 3, N_max: 100, k: sym.ln(2)}),
    Logistic_growth_solution.rhs.subs({N0: 30, N_max: 100, k: sym.ln(2)}),
    Logistic_growth_solution.rhs.subs({N0: 130, N_max: 100, k: sym.ln(2)}),
    (t, 0, 15),
    xlabel="t",
    ylabel="N(t)",
    axis_center=(0,0),
    ylim=(0,150),
);

**Lets calculate substrate disappearance in the presense of Michaelis-Menten enzyme:**

## $\frac{d[S](t)}{dt}=-\frac {V_{max} \cdot \frac{[S](t)}{K_m}}{1+\frac{[S](t)}{K_m}}$  
  
where $t$ is time,  $V_{max}$ is maximal enzyme rate,  $[S](t)$ is substrate concentration of drug at time $t$, $K_m$ is Michaelis-Menten constant and $[S]_0$ is substrate concentration at $t=0$. 

In [ ]:
S = sym.symbols("S", cls=sym.Function)
t, V_max, S_0, K_m = sym.symbols("t, V_max, S_0, K_m")
MM_enzyme_solution = sym.dsolve(sym.diff(S(t), t) + V_max * (S(t)/K_m)/(1 + S(t)/K_m),
                                S(t),
                                ics={S(0): S_0},
                                #simplify=False   #try adding and removing this option to see how the result changes
                               )
MM_enzyme_solution.simplify()

If you include "simplify=False" argument from dsolve() above then the result will be represented without using W() where W() is called LambertW function and is defined as x = W(y) when x * e^x = y

In [ ]:
sym.plot(
    MM_enzyme_solution.rhs.subs({S_0: 1, V_max: 1, K_m: 1}),
    MM_enzyme_solution.rhs.subs({S_0: 2, V_max: 1, K_m: 1}),
    MM_enzyme_solution.rhs.subs({S_0: 4, V_max: 1, K_m: 1}),
    (t, 0, 5),
    xlabel="t",
    ylabel="[S](t)"
);

**Lets calculate [ES] concentration over time without Michaelis-Menten approximation:**  
  
$E + S  \overset{k_{on}}{ \underset{k_{off}}{\rightleftarrows}} ES \xrightarrow{k_{cat}} E + P$  
  
$\frac{d[ES]}{dt}=k_{on}[E][S]-(k_{off}+k_{cat})[ES]$  
  
Michaelis and Menten (Haldane used this derivation to be precise) assumed that $\frac{d[ES]}{dt}=0$ and used $[E]_{total}=[E]+[ES]$ to derive Michaelis-Menten equation:
  
$\frac{d[P](t)}{dt}=k_{cat}[ES]=\frac {V_{max} \cdot \frac{[S](t)}{K_m}}{1+\frac{[S](t)}{K_m}}$  
  
where $t$ is time,  $V_{max}=k_{cat}[E]_{total}$ is maximal enzyme rate,  $[S](t)$ is substrate concentration of drug at time $t$, $K_m=\frac{k_{off}+k_{cat}}{k_{on}}$ is Michaelis-Menten constant.  
  
Let's assume that $\frac{d[ES]}{dt} \neq 0$ and calculate $[ES](t)$ by solving the differential equation:  
  
$\frac{d[ES](t)}{dt}=k_{on}[E][S]-(k_{off}+k_{cat})[ES](t)=k_{on}([E]_{total}-[ES](t))[S]-(k_{off}+k_{cat})[ES](t)$  
  
We will assume that $S$ is constant becasue $\frac{d[ES](t)}{dt}$ will become $0$ before $S$ will change.

In [ ]:
ES = sym.symbols("ES", cls=sym.Function,nonnegative=True)
kon, koff, kcat  = sym.symbols("k_{on}, k_{off}, k_{cat}",positive=True)
Etot, S, t = sym.symbols("E_{total}, S, t",nonnegative=True)

C1, C2 = sym.symbols("C1 C2")

ES_solution = sym.dsolve(
    sym.Eq(ES(t).diff(t), kon*(Etot - ES(t))*S - (koff + kcat)*ES(t)),
    ES(t),
    ics={ES(0): 0}
)

ES_solution.simplify()

In [ ]:
kon_value =1**6
koff_value =10
kcat_value =10
sym.plot(
    ES_solution.rhs.subs({Etot: 1, kon:kon_value, koff:koff_value, kcat:kcat_value, S:0.1*(koff_value + kcat_value)/kon_value}),
    (t, 0, 1),
    xlabel="t",
    ylabel="[ES](t)"
);

Michaelis-Menten steady state approximation seems valid.  
Let's compare the kinetics of $[ES]$ coming to steady state with kinetics of $S$ disappearence that we determined above using the same parameters:

In [ ]:
S = sym.symbols("S", cls=sym.Function)
t, V_max, S_0, K_m = sym.symbols("t, V_max, S_0, K_m")
MM_enzyme_solution = sym.dsolve(sym.diff(S(t), t) + V_max * (S(t)/K_m)/(1 + S(t)/K_m),
                                S(t),
                                ics={S(0): S_0},
                                #simplify=False   #try adding and removing this option to see how the result changes
                               )
MM_enzyme_solution.simplify()

sym.plot(
    MM_enzyme_solution.rhs.subs({S_0: 20, V_max: 1, K_m: 2}),
    (t, 0, 1),
    xlabel="t",
    ylabel="[S](t)",
    axis_center=(0,0),
    ylim=(0,20),
);

Our assumtion that $S$ is constant while $ES$ is coming to steady state is reasonable as at $t=0.5$ $ES$ mostly reached steady state but $S$ barely decreased from 20.